In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import mir_eval.transcription
from model import get_model

from src.common import MaestroSplitType, MidiWrapper, TimeKeeper
from src.maestro2 import MaestroDatasetSplit

In [ ]:
dataset = MaestroDatasetSplit(MaestroSplitType.VALIDATION)

In [ ]:
entry = dataset.split.entries[1]
start_ms = 1000.0
end_ms = 15000.0

In [ ]:
mel = entry.load_audio.compute_log_mel_spectrogram()

In [ ]:
from torch import tensor

mel_part = mel[:, TimeKeeper.ms_to_frames(start_ms):TimeKeeper.ms_to_frames(end_ms)]
mel_t = tensor(mel_part).unsqueeze(0).unsqueeze(0).to('cuda')

In [ ]:
model = get_model(21, 'cuda')

In [ ]:
result_t = model.forward(mel_t[:, :, :21, :])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
entry.load_audio.display_ipython(start_ms / 1000., end_ms / 1000.)
entry.load_midi.plot_piano_roll_chunk(start_ms / 1000., end_ms / 1000.)
plt.figure(figsize=(int(end_ms - start_ms) / 1000, 8))
plt.imshow((mel_t[0, 0, :, :] > 3).detach().cpu(), cmap='gray', aspect='auto', origin='lower', interpolation='none')
plt.xlabel('Time')
plt.ylabel('Mel bin')
plt.colorbar()
None

In [ ]:
from src.common import MidiWrapper
midiw = MidiWrapper.from_piano_roll(mel_t[0, 0, :, :].cpu().detach().numpy(), on_threshold=2)

In [ ]:
mel_t[0, 0, :128, :].cpu().detach().numpy().shape

In [ ]:
midiw.plot_piano_roll()

In [ ]:
import numpy as np

np.any(midiw.midi.get_piano_roll() > 0)

In [ ]:
midiw.midi.instruments[0].notes

In [ ]:
roll2 = dataset.split.entries[0].load_midi.get_piano_roll()[:, 1000:1100]
roll2

In [ ]:
midiw = MidiWrapper.from_piano_roll(roll2, on_threshold=0.502)


In [ ]:
midiw.plot_piano_roll()